<a href="https://colab.research.google.com/github/IsaacFigNewton/Analyzing-Hate-Crime-Data/blob/main/Hate_Crime_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODO:
####Fix the excessive number of bias/target groups
####Add the ethnicity_race_cols to demo_df

#Import Stuff

###Import all libraries

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import re

###Import datasets

In [2]:
crime_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/hate_crime/hate_crime.csv", on_bad_lines='skip')
city_demo_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/demographics/city/ACSST1Y2022.S0101-Data.csv", on_bad_lines='skip')
county_demo_df = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/demographics/county/ACSDP1Y2022.DP05-Data.csv", on_bad_lines='skip')

#Data Cleaning

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
#only consider 2022 crime data from cities and counties
crime_df = crime_df[(crime_df['data_year'] == 2022) & ((crime_df['agency_type_name'] == "City") | (crime_df['agency_type_name'] == "County"))]

In [5]:
# use the entries of the first row as the column headers for easier management
def fixHeaders(df):
  new_headers = df.iloc[0]
  new_df = df[1:]
  new_df.columns = new_headers
  return new_df

In [6]:
#fix the headers
city_demo_df = fixHeaders(city_demo_df)
county_demo_df = fixHeaders(county_demo_df)

In [7]:
def split_area_name(area):
    result = [np.nan, np.nan, np.nan]
    if ", " in area:
        result = area.split(", ") + [np.nan]
    if " city" in result[0].lower():
        result[0] = result[0][0:-5]
        result[2] = result[1]
        result[1] = "City"
    elif " county" in result[0].lower():
        result[0] = result[0][0:-7]
        result[2] = result[1]
        result[1] = "County"

    return result + [np.nan] * (3 - len(result))

def splitArea(df):
    df[["pug_agency_name", "agency_type_name", "state_name"]] = df["Geographic Area Name"].map(split_area_name).apply(pd.Series)

In [8]:
splitArea(city_demo_df)
splitArea(county_demo_df)

# city_demo_df.head()

In [9]:
def split_incident_date(date):
    result = date.split("-")

    return result + [np.nan] * (3 - len(result))

crime_df[["data_year", "incident_month", "incident_day"]] = crime_df["incident_date"].map(split_incident_date).apply(pd.Series)

In [10]:
def split_bias_desc(desc):
  # break up the biases into individual biases
  biases = desc.split(";")
  for i in range(len(biases)):
      biases[i] = biases[i]\
                    .replace("Anti-", "")\
                    .replace("(Male)", "")\
                    .replace("(Female)", "")\
                    .replace("Lesbian, Gay, Bisexual, or Transgender (Mixed Group)", "LGBTQ")\
                    .strip()

  # add placeholders
  biases = biases + [np.nan] * (5 - len(biases))

  return biases

# TODO: find max bias count in mapping function above?
max_bias_count = 5

bias_cols = ["bias" + str(i) for i in range(max_bias_count)]
crime_df[bias_cols] = crime_df["bias_desc"]\
                                                            .map(split_bias_desc)\
                                                            .apply(pd.Series)

In [11]:
#create a list of all biases
all_biases = []
for col in bias_cols:
  all_biases.extend(crime_df[col].unique())
unique_biases = set(all_biases)

crime_df.iloc[0]

incident_id                                                               1476719
data_year                                                                    2022
ori                                                                     AK0010500
pug_agency_name                                                            Kodiak
pub_agency_unit                                                               NaN
agency_type_name                                                             City
state_abbr                                                                     AK
state_name                                                                 Alaska
division_name                                                             Pacific
region_name                                                                  West
population_group_code                                                           6
population_group_description                         Cities from 2,500 thru 9,999
incident_date   

In [28]:
# create dummy columns for all hate crime biases
dummies = pd.get_dummies(list(unique_biases))

# combine the dummy columns with crime_df
crime_df = pd.concat([crime_df, dummies], axis=1)

# set all dummy values to 0
# for col in dummies.columns:
#     crime_df[col].values[:] = 0
crime_df.loc[:, dummies.columns] = 0

crime_df.head()

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias,incident_month,incident_day,bias1,bias2,bias3,bias4,bias5,American Indian or Alaska Native,Arab,Asian,Atheism/Agnosticism,Bisexual,Black or African American,Buddhist,Catholic,Church of Jesus Christ,"Eastern Orthodox (Russian, Greek, Other)",Female,Gay,Gender Non-Conforming,Heterosexual,Hindu,Hispanic or Latino,Islamic (Muslim),Jehovah's Witness,Jewish,LGBTQ,Lesbian,Male,Mental Disability,"Multiple Races, Group","Multiple Religions, Group",Native Hawaiian or Other Pacific Islander,Other Christian,Other Race/Ethnicity/Ancestry,Other Religion,Physical Disability,Protestant,Sikh,Transgender,White,NaN,American Indian or Alaska Native,Arab,Asian,Atheism/Agnosticism,Bisexual,Black or African American,Buddhist,Catholic,Church of Jesus Christ,"Eastern Orthodox (Russian, Greek, Other)",Female,Gay,Gender Non-Conforming,Heterosexual,Hindu,Hispanic or Latino,Islamic (Muslim),Jehovah's Witness,Jewish,LGBTQ,Lesbian,Male,Mental Disability,"Multiple Races, Group","Multiple Religions, Group",Native Hawaiian or Other Pacific Islander,Other Christian,Other Race/Ethnicity/Ancestry,Other Religion,Physical Disability,Protestant,Sikh,Transgender,White,American Indian or Alaska Native,Arab,Asian,Atheism/Agnosticism,Bisexual,Black or African American,Buddhist,Catholic,Church of Jesus Christ,"Eastern Orthodox (Russian, Greek, Other)",Female,Gay,Gender Non-Conforming,Heterosexual,Hindu,Hispanic or Latino,Islamic (Muslim),Jehovah's Witness,Jewish,LGBTQ,Lesbian,Male,Mental Disability,"Multiple Races, Group","Multiple Religions, Group",Native Hawaiian or Other Pacific Islander,Other Christian,Other Race/Ethnicity/Ancestry,Other Religion,Physical Disability,Protestant,Sikh,Transgender,White
230020,1476719.0,2022,AK0010500,Kodiak,NaN,City,AK,Alaska,Pacific,West,6,"Cities from 2,500 thru 9,999",2022-07-31,1.0,0.0,1.0,1.0,0.0,American Indian or Alaska Native,Not Specified,1.0,Destruction/Damage/Vandalism of Property,1.0,Residence/Home,"Anti-Lesbian, Gay, Bisexual, or Transgender (M...",Individual,S,S,07,31,LGBTQ,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
230021,1483026.0,2022,AK0015600,Wasilla,NaN,City,AK,Alaska,Pacific,West,6,"Cities from 2,500 thru 9,999",2022-10-06,0.0,0.0,0.0,NaN,NaN,Not Specified,Not Specified,1.0,Destruction/Damage/Vandalism of Property,0.0,Church/Synagogue/Temple/Mosque,Anti-Other Christian,Religious Organization,S,S,10,06,Other Christian,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
230022,1484192.0,2022,AK0010100,Anchorage,NaN,City,AK,Alaska,Pacific,West,1C,"Cities from 250,000 thru 499,999",2022-03-25,1.0,0.0,0.0,0.0,0.0,Unknown,Unknown,1.0,Intimidation,1.0,Community Center,Anti-Jewish,Individual,S,S,03,25,Jewish,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
230023,1484193.0,2022,AK0010100,Anchorage,NaN,City,AK,Alaska,Pacific,West,1C,"Cities from 250,000 thru 499,999",2022-06-30,NaN,NaN,0.0,0.0,0.0,Unknown,Unknown,1.0,Destruction/Damage/Vandalism of Property,NaN,Church/Synagogue/Temple/Mosque,Anti-Other Christian,Religious Organization,S,S,06,30,Other Christian,NaN,NaN

In [ ]:
# for each entry
for i in range(len(crime_df.index)):
    # for each bias column
    for j in range(1, 6):
        bias = crime_df.iloc[i]["bias" + str(j)]
        # print(bias)
        if bias in unique_biases:
            # Use the iloc method to access the DataFrame by row and column indices
            crime_df.at[i, bias] = 1

# # Iterate over bias columns
# for j in range(1, 6):
#     bias_col = "bias" + str(j)
#     bias = crime_df[bias_col]
#     crime_df.loc[crime_df[bias_col].isin(unique_biases), bias] = np.where(crime_df[bias_col].isin(unique_biases), 1, 0)

In [ ]:
crime_df.iloc[10]

In [ ]:
def fix_column_names(df):
  removalList = {"SEX AND AGE!!", "SEX AND ", "Estimate!!", "Total!!", "Total population!!", "CITIZEN, VOTING AGE POPULATION!!", "AGE!!"}

  new_cols = []
  for col in df.columns:
    col = str(col)
    for term in removalList:
      if term in col:
        col = col.replace(term, "")
    new_cols.append(col)

  return new_cols

county_demo_df.columns = fix_column_names(county_demo_df)
city_demo_df.columns = fix_column_names(city_demo_df)

In [ ]:
#combine the city_df age-related columns as needed to merge better with the county_demo_df
def combine_city_df_cols(df_s0101):
    formatted_df = df_s0101.copy(deep=True)

    formatted_df['25 to 34 years'] = df_s0101['25 to 29 years'] + df_s0101['30 to 34 years']
    formatted_df['35 to 44 years'] = df_s0101['35 to 39 years'] + df_s0101['40 to 44 years']
    formatted_df['45 to 54 years'] = df_s0101['45 to 49 years'] + df_s0101['50 to 54 years']
    formatted_df['65 to 74 years'] = df_s0101['65 to 69 years'] + df_s0101['70 to 74 years']
    formatted_df['75 to 84 years'] = df_s0101['75 to 79 years'] + df_s0101['80 to 84 years']

    return formatted_df

city_demo_df = combine_city_df_cols(city_demo_df)
# city_demo_df

In [ ]:
# fix any missing values
city_demo_df = city_demo_df.replace("(X)", np.nan)
county_demo_df = county_demo_df.replace("(X)", np.nan)

###Combine the datasets

In [ ]:
ethnicity_race_cols = pd.read_csv("https://raw.githubusercontent.com/IsaacFigNewton/Analyzing-Hate-Crime-Data/main/demographics/county/ethnicity_race_col_names")["One race"]
ethnicity_race_cols

In [ ]:
# TODO: add the ethnicity_race_cols to demo_df

In [ ]:
# Create new columns that combine data shared between all the city and county entries
overlapping_columns = set(city_demo_df.columns).intersection(set(county_demo_df.columns))

# include overlapping data and race and ethnicity data from the county dataset
print(overlapping_columns.union(ethnicity_race_cols))
demo_df = pd.concat([city_demo_df[list(overlapping_columns)], county_demo_df[list(overlapping_columns)]])

In [ ]:
# combine the crime and population datasets
merged_df = pd.merge(crime_df, demo_df, on=["pug_agency_name", "agency_type_name", "state_name"])

###Clean the merged dataset

In [ ]:
merged_df.drop(["ori", "state_abbr", "population_group_code", "pub_agency_unit",
                "nan", "Geography", "Percent!!Total population", "population_group_description",
                "Geographic Area Name", "incident_date", "pub_agency_unit"], axis=1, inplace=True)
merged_df.drop(list(merged_df.filter(regex = "Margin of Error")), axis = 1, inplace = True)

In [ ]:
# Convert object columns to string dtype
object_columns = ["pug_agency_name", "agency_type_name", "state_name", "division_name", "offender_race",
                  "offender_ethnicity", "offense_name", "location_name", "bias_desc", "victim_types", "multiple_offense",
                  "multiple_bias"]
merged_df[object_columns] = merged_df[object_columns].astype(str)

# Number columns
int_columns = ["Total population", "Under 5 years", "5 to 9 years", "45 to 54 years", "10 to 14 years", "75 to 84 years", "60 to 64 years",
                      "25 to 34 years", "15 to 19 years", "20 to 24 years", "35 to 44 years", "55 to 59 years", "65 to 74 years",
                      "85 years and over", "incident_month", "incident_day"] + list(unique_biases)
merged_df[int_columns] = merged_df[int_columns].astype(int)

# Convert columns with percentage values to float
percentage_columns = ["Percent!!60 to 64 years", "Percent!!15 to 19 years", "Percent!!5 to 9 years", "Percent!!10 to 14 years",
                      "Percent!!Under 5 years", "Percent!!20 to 24 years", "Percent!!55 to 59 years", "Percent!!85 years and over"]
merged_df[percentage_columns] = merged_df[percentage_columns].astype(float)

merged_df.dtypes

# Explore the Data

##Individual Datasets

###Hate Crime Dataset

In [ ]:
crime_df.head()

In [ ]:
unique_biases

In [ ]:
crime_df.groupby('region_name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
crime_df.groupby('agency_type_name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
crime_df['total_offender_count'].plot(kind='hist', bins=20, title='total_offender_count')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
crime_df.groupby('offender_race').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#TODO: Fix the excessive number of bias/target groups
# create a list of all individual biases
# create new dummy columns for each bias that hold boolean values, set to False by default
# parse the bias columns' truth values from the bias_desc column

In [ ]:
crime_df.groupby('bias1').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
crime_df['adult_victim_count'].plot.hist(bins=20, title='adult_victim_count')

In [ ]:
crime_df['juvenile_victim_count'].plot(kind='hist', bins=20, title='juvenile_victim_count')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['offender_race'].value_counts()
    for x_label, grp in crime_df.groupby('region_name')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('region_name')
_ = plt.ylabel('offender_race')

###City Demographics Dataset

In [ ]:
city_demo_df.head()

###County Demographics Dataset

In [ ]:
county_demo_df.head()

##Merged Dataset

In [ ]:
merged_df.head()

In [ ]:
#convert the categorical columns to numerical ones and store the modified df as a new df for correlation analysis
categorical_to_int_df = merged_df
# Select only the numerical columns
numerical_cols = categorical_to_int_df.select_dtypes(include=['int64', 'float64', np.number]).drop(labels=["incident_id"], axis=1)
# numerical_cols

In [ ]:
# Create a correlation matrix
corr = merged_df[numerical_cols.columns].corr()

# Create a heatmap of the correlation matrix
fig, ax = plt.subplots(figsize=(30,20))
sns.heatmap(corr, annot=True, ax=ax)

In [ ]:
merged_df["color"] = merged_df["region_name"].map({
    "Midwest": "red",
    "West": "yellow",
    "Northeast": "green",
    "South": "blue"
})
merged_df["size"] = merged_df.groupby(["incident_month", "region_name"]).count().reset_index()["incident_id"]

#remove the max row limit for altair
alt.data_transformers.disable_max_rows()

alt.Chart(merged_df.dropna()).mark_circle().encode(
    x="total_offender_count",
    y="total_individual_victims",
    color=alt.Color("color", scale=None),
    size="size"
)